# Importing Libraries

In [121]:
import numpy as np
import pandas as pd
import math
import re
import nltk
import string
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer
from scipy.stats import randint
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import f1_score, make_scorer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import f1_score, make_scorer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.linear_model import LogisticRegression

In [163]:
# @title Fill the form and run it before proceeding
language = 'Bodo' # @param ["Assamese","Bodo", "Bengali", "Sinhala"]
model_name = 'XGB' # @param ["Logistic Regression","Support Vector Machine","Decision Tree","XGB"]
embedding = 'count vectorizer' # @param ["count vectorizer","tfidf vectorizer"]
ngrams = '1-2' # @param ["1-2","1-3"]
train_test_split_ = True # @param {type:"boolean"}
n_iter = 1 # @param {type:"slider", min:1, max:500, step:1}


# Dataset upload

Train data upload

In [123]:
from google.colab import files

uploaded_file = files.upload()
uploaded_filename = list(uploaded_file.keys())[0]

Saving train_BO_AH_HASOC2023(1).csv to train_BO_AH_HASOC2023(1) (2).csv


Test data upload

In [ ]:
print("please upload test data file")
test_file = files.upload()
test_filename = list(test_file.keys())[0]

# Randomized Search Class

In [164]:
class RandomSearch:
  def __init__(self,language,model_name,uploaded_filename,embedding,ngrams,train_test_split_,n_iter):
    self.language = language
    self.model_name = model_name
    self.uploaded_filename = uploaded_filename
    self.df = pd.read_csv(self.uploaded_filename)
    self.embedding = embedding
    self.ngrams = ngrams
    self.vec = ""
    self.train_test_split_ = train_test_split_
    self.n_iter = n_iter
    self.x_train = []
    self.y_train = []
    self.x_test = []
    self.y_test = []
  def username_remover(self,text):
    text = re.sub(r'@[^ ]+',"", text)
    return text
  def clean(self,text):
    text = re.sub(r'#\w+', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    pattern = re.compile(r"(.)\1{2,}")
    words = text.split()
    reduced_words = []
    for word in words:
        reduced_word = pattern.sub(r"\1\1", word)
        reduced_words.append(reduced_word)
    text = ' '.join(reduced_words)
    return text
  def fitter(self,x_train,x_test,ngrams):
    if (ngrams=="1-2"):
      ngram_range = (1,2)
    else:
      ngram_range = (1,3)
    cv = CountVectorizer(ngram_range=ngram_range)
    # cv.fit(x_train)
    x_train_final = cv.fit_transform(x_train)
    self.vec = cv
    x_test_final = cv.transform(x_test)
    print(len(cv.vocabulary_))
    return [x_train_final,x_test_final]
  def fitter2(self,x_train,x_test,ngrams):
    if (ngrams=="1-2"):
      ngram_range = (1,2)
    else:
      ngram_range = (1,3)
    cv = TfidfVectorizer(ngram_range=ngram_range)
    # cv.fit(x_train)
    x_train_final = cv.fit_transform(x_train)
    self.vec = cv
    x_test_final = cv.transform(x_test)
    print(len(cv.vocabulary_))
    return [x_train_final,x_test_final]
  def split(self):
    # self.cleaner()
    total = len(self.df["text"].to_list())
    self.x_train,self.x_test, self.y_train,self.y_test = train_test_split(self.df["text"], self.df["task_1"],stratify=self.df["task_1"], test_size=2/total, random_state=42)
    if self.embedding == "count vectorizer":
      result = self.fitter(self.x_train,self.x_test,self.ngrams)
      self.x_train = result[0]
      self.x_test = result[1]
    else:
      result = self.fitter2(self.x_train,self.x_test,self.ngrams)
      self.x_train = result[0]
      self.x_test = result[1]


  def cleaner(self):
    self.df["text"] = self.df["text"].apply(self.username_remover)
    self.df["text"] = self.df["text"].apply(self.clean)
    self.df["task_1"] = self.df["task_1"].map({"NOT" : 0, "HOF":1})
    self.df = self.df.drop_duplicates('text')
    return self.df["text"]
  def randomsearch_decisiontree(self,n_iter):
    X = self.x_train
    y = self.y_train
    param_dist = {
        'decision_tree__criterion': ['gini', 'entropy'],
        'decision_tree__splitter': ['best', 'random'],
        'decision_tree__max_depth': randint(1, 100),
        'decision_tree__min_samples_split': randint(2, 11),
        'decision_tree__min_samples_leaf': randint(1, 11)
    }
    dt_classifier = DecisionTreeClassifier()
    n_components_range = np.arange(1, X.shape[1]+1)
    param_dist['lsa__n_components'] = n_components_range

    lsa = TruncatedSVD(random_state=42)
    pipeline = Pipeline([('lsa', lsa), ('decision_tree', dt_classifier)])

    f1_scorer = make_scorer(f1_score, average='macro')
    random_search = RandomizedSearchCV(
        pipeline, param_distributions=param_dist, n_iter=n_iter, cv=5, random_state=42,
        n_jobs=-1, scoring=f1_scorer
    )
    random_search.fit(X, y)
    return random_search.best_params_, random_search.best_score_
  def random_search_logistic_regression(self,n_iter):
    X = self.x_train
    y = self.y_train
    param_dist = {
        'logistic_regression__penalty': ['l1', 'l2'],
        'logistic_regression__C': np.logspace(-4, 4, 100),
        'logistic_regression__solver': ['liblinear', 'saga'],
        'logistic_regression__max_iter': randint(100, 1000),
    }

    logistic_regression = LogisticRegression()
    n_components_range = np.arange(1, X.shape[1] + 1)
    param_dist['lsa__n_components'] = n_components_range

    lsa = TruncatedSVD(random_state=42)
    pipeline = Pipeline([('lsa', lsa), ('logistic_regression', logistic_regression)])

    f1_scorer = make_scorer(f1_score, average='macro')
    random_search = RandomizedSearchCV(
        pipeline, param_distributions=param_dist, n_iter=n_iter, cv=5, random_state=42,
        n_jobs=-1, scoring=f1_scorer
    )
    random_search.fit(X, y)
    return random_search.best_params_, random_search.best_score_
  def random_search_svc(self, n_iter):
    X = self.x_train
    y = self.y_train
    param_dist = {
        'svc__C': np.logspace(-4, 4, 100),
        'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'svc__gamma': ['scale', 'auto'] + list(np.logspace(-4, 3, 8)),
    }

    svc_classifier = SVC()
    n_components_range = np.arange(1, X.shape[1] + 1)
    param_dist['lsa__n_components'] = n_components_range

    lsa = TruncatedSVD(random_state=42)
    pipeline = Pipeline([('lsa', lsa), ('svc', svc_classifier)])

    f1_scorer = make_scorer(f1_score, average='macro')
    random_search = RandomizedSearchCV(
        pipeline, param_distributions=param_dist, n_iter=n_iter, cv=5, random_state=42,
        n_jobs=-1, scoring=f1_scorer
    )
    random_search.fit(X, y)
    return random_search.best_params_, random_search.best_score_
  def random_search_xgboost(self, n_iter):
    X = self.x_train
    y = self.y_train
    param_dist = {
        'xgb__learning_rate': np.logspace(-4, 0, 100),
        'xgb__n_estimators': randint(50, 200),
        'xgb__max_depth': randint(1, 20),
        'xgb__min_child_weight': randint(1, 10),
        'xgb__subsample': uniform(0.6, 0.4),
        'xgb__colsample_bytree': uniform(0.6, 0.4),
    }

    xgb_classifier = XGBClassifier(random_state=42, eval_metric='mlogloss')
    n_components_range = np.arange(1, X.shape[1] + 1)
    param_dist['lsa__n_components'] = n_components_range

    lsa = TruncatedSVD(random_state=42)
    pipeline = Pipeline([('lsa', lsa), ('xgb', xgb_classifier)])

    f1_scorer = make_scorer(f1_score, average='macro')
    random_search = RandomizedSearchCV(
        pipeline, param_distributions=param_dist, n_iter=n_iter, cv=5, random_state=42,
        n_jobs=-1, scoring=f1_scorer
    )
    random_search.fit(X, y)
    return random_search.best_params_, random_search.best_score_
  def get_results(self):
    if model_name == "Logistic Regression":
      results = self.random_search_logistic_regression(self.n_iter)
    elif model_name == "XGB":
      results = self.random_search_xgboost(self.n_iter)
    elif model_name == "Decision Tree":
      results = self.randomsearch_decisiontree(self.n_iter)
    elif model_name == "Support Vector Machine":
      results = self.random_search_svc(self.n_iter)
    return results








In [165]:
searcher = RandomSearch(language,model_name,uploaded_filename,embedding,ngrams,train_test_split_,n_iter)
print(searcher.cleaner())
searcher.split()

0            गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                 निखावरि सुबुंफोरा सिखाव
2              मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                 थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4       माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खा...
                              ...                        
1674    नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिद...
1675    एै मावजि लाब गैया दानो बनद खालामनायनि खोथा बुं...
1676                            सिखला फुरकव रपे खलामनांगव
1677                        सेनदेल खुबै नांगौलै सालाफोरखौ
1678    राकेस बोरमा सिखावजोंनो थाफाहैदोलै नोंबो आदै जा...
Name: text, Length: 1679, dtype: object
4129


In [166]:
best_params = searcher.get_results()
print(best_params)

({'lsa__n_components': 861, 'xgb__colsample_bytree': 0.6733739159464655, 'xgb__learning_rate': 0.07390722033525783, 'xgb__max_depth': 7, 'xgb__min_child_weight': 3, 'xgb__n_estimators': 124, 'xgb__subsample': 0.7836995567863468}, 0.638511081263039)


In [167]:
best_params

({'lsa__n_components': 861,
  'xgb__colsample_bytree': 0.6733739159464655,
  'xgb__learning_rate': 0.07390722033525783,
  'xgb__max_depth': 7,
  'xgb__min_child_weight': 3,
  'xgb__n_estimators': 124,
  'xgb__subsample': 0.7836995567863468},
 0.638511081263039)

In [168]:
copy_of_best_params = best_params

In [169]:
copy_of_best_params[0]["lsa__n_components"]

861

In [170]:
class Tester:
  def __init__(self,test_df,language,model_name,embedding,ngrams,model,lsa,vectorizer):
    self.df = test_df
    self.embedding = embedding
    self.ngrams = ngrams
    self.model = model
    self.language = language
    self.model_name = model_name
    self.csv_filename = ""
    self.lsa = lsa
    self.vectorizer = vectorizer
  def username_remover(self,text):
    text = re.sub(r'@[^ ]+',"", text)
    return text
  def clean(self,text):
    text = re.sub(r'#\w+', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    pattern = re.compile(r"(.)\1{2,}")
    words = text.split()
    reduced_words = []
    for word in words:
        reduced_word = pattern.sub(r"\1\1", word)
        reduced_words.append(reduced_word)
    text = ' '.join(reduced_words)
    return text
  def cleaner(self):
    self.df["text"] = self.df["text"].apply(self.username_remover)
    self.df["text"] = self.df["text"].apply(self.clean)
    # self.df["task_1"] = self.df["task_1"].map({"NOT" : 0, "HOF":1})
    self.df = self.df.drop_duplicates('text')
    return self.df["text"]
  def save_predictions(self):
    x_test = self.vectorizer.transform(self.df["text"])
    x_test_lsa = self.lsa.transform(x_test)
    predictions = self.model.predict(x_test_lsa)
    self.df["task_1"] = list(predictions)
    self.df = self.df.drop("text", axis=1)
    csv_filename = str(self.language)+"_"+str(self.model_name)+"_"+str(embedding)+"_"+str(ngrams)+".csv"
    self.csv_filename = csv_filename
    self.df.to_csv(csv_filename, index=False)


In [171]:
if model_name == "Logistic Regression":
  lsa = TruncatedSVD(n_components=copy_of_best_params[0]["lsa__n_components"])
  X_train_lsa = lsa.fit_transform(searcher.x_train)

  model = LogisticRegression(
      C=copy_of_best_params[0]["logistic_regression__C"],
      max_iter=copy_of_best_params[0]["logistic_regression__max_iter"],
      penalty=copy_of_best_params[0]["logistic_regression__penalty"],
      solver=copy_of_best_params[0]["logistic_regression__solver"]
  )
  model.fit(X_train_lsa, searcher.y_train)
  # print("please upload test data file")
  # test_file = files.upload()
  # test_filename = list(test_file.keys())[0]
  test_df = pd.read_csv(test_filename)
  vectorizer_test = searcher.vec
  tester = Tester(test_df,language,model_name,embedding,ngrams,model,lsa,vectorizer_test)
  tester.cleaner()
  tester.save_predictions()
elif model_name == "XGB":
  lsa = TruncatedSVD(n_components=copy_of_best_params[0]["lsa__n_components"])
  X_train_lsa = lsa.fit_transform(searcher.x_train)

  model = XGBClassifier(
    n_estimators=copy_of_best_params[0]["xgb__n_estimators"],
    max_depth=copy_of_best_params[0]["xgb__max_depth"],
    learning_rate=copy_of_best_params[0]["xgb__learning_rate"],
    min_child_weight = copy_of_best_params[0]["xgb__min_child_weight"],
    subsample = copy_of_best_params[0]["xgb__subsample"],
    colsample_bytree = copy_of_best_params[0]["xgb__colsample_bytree"]
  )
  model.fit(X_train_lsa, searcher.y_train)
  # print("please upload test data file")
  # test_file = files.upload()
  # test_filename = list(test_file.keys())[0]
  test_df = pd.read_csv(test_filename)
  vectorizer_test = searcher.vec
  tester = Tester(test_df,language,model_name,embedding,ngrams,model,lsa,vectorizer_test)
  tester.cleaner()
  tester.save_predictions()
elif model_name == "Decision Tree":
  lsa = TruncatedSVD(n_components=copy_of_best_params[0]["lsa__n_components"])
  X_train_lsa = lsa.fit_transform(searcher.x_train)

  model = DecisionTreeClassifier(
    criterion=copy_of_best_params[0]['decision_tree__criterion'],
    splitter=copy_of_best_params[0]['decision_tree__splitter'],
    max_depth=copy_of_best_params[0]['decision_tree__max_depth'],
    min_samples_split=copy_of_best_params[0]['decision_tree__min_samples_split'],
    min_samples_leaf=copy_of_best_params[0]['decision_tree__min_samples_leaf']
  )
  model.fit(X_train_lsa, searcher.y_train)
  # print("please upload test data file")
  # test_file = files.upload()
  # test_filename = list(test_file.keys())[0]
  test_df = pd.read_csv(test_filename)
  vectorizer_test = searcher.vec
  tester = Tester(test_df,language,model_name,embedding,ngrams,model,lsa,vectorizer_test)
  tester.cleaner()
  tester.save_predictions()
elif model_name == "Support Vector Machine":
  lsa = TruncatedSVD(n_components=copy_of_best_params[0]["lsa__n_components"])
  X_train_lsa = lsa.fit_transform(searcher.x_train)

  model = SVC(
    C=copy_of_best_params[0]['svc__C'],
    kernel=copy_of_best_params[0]['svc__kernel'],
    gamma=copy_of_best_params[0]['svc__gamma']
  )
  model.fit(X_train_lsa, searcher.y_train)
  # print("please upload test data file")
  # test_file = files.upload()
  # test_filename = list(test_file.keys())[0]
  test_df = pd.read_csv(test_filename)
  vectorizer_test = searcher.vec
  tester = Tester(test_df,language,model_name,embedding,ngrams,model,lsa,vectorizer_test)
  tester.cleaner()
  tester.save_predictions()





please upload test data file


Saving test_BO_AH_HASOC2023(1).csv to test_BO_AH_HASOC2023(1) (7).csv


In [172]:
test_df

,S. No.,text
0,1,BPF बानाय लांनाय लामाया 5 बोसोरानो जोरासै
1,2,बै समाव माबेयाव हाबसोनानै दंमोन नोंलाय दाना बु...
2,3,बे थांखिखौ मिनिग्रापोरा हारिखौ लेवारपोरबायदि थ...
3,4,मोसौ खुगायाव एमफौ नांबाय नोंनाव सैमा
4,5,2003आव BTC गोरोबथा जादों बेनि थाखाय बो थोजासे ...
...,...,...
415,416,आं आनो खाजा होआखै मोन
416,417,बियो आंखौ बिनि बिमा बिफा बुथारनायनि थाखाय दायन...
417,418,राहुलआ गावनि फोरोंगिरिखौ मान होआ
418,419,राकेशआ गावनि फोरोंगिरिखौ मान होआ


In [173]:
df_res = pd.read_csv(tester.csv_filename)
df_res["task_1"].value_counts()

1    304
0    116
Name: task_1, dtype: int64